In [ ]:
# %pip install requests

In [41]:
# Importar las librerías necesarias
import requests
import os
import re

In [42]:
# Definir la ruta base del proyecto
BASE_DIR = os.getcwd()
print(f"Directorio base: {BASE_DIR}")

Directorio base: c:\Users\alexx\ia_identificar_medicamentos


In [43]:
# Ejeceutar Tesseract OCR para extraer el texto de la imagen
# ruta_tesseract = "D:/Tesseract/tesseract.exe"
# ruta_imagen = "D:/Universidad/Proyecto_Odiseia4good/Fotos/foto.jpg"
# ruta_salida = "D:/Universidad/Proyecto_Odiseia4good/Fotos/resultado"

# comando = f'"{ruta_tesseract}" "{ruta_imagen}" "{ruta_salida}" --psm 11'
# print(f"Ejecutando: {comando}")

# subprocess.run(comando, shell=True)
# print("Comando ejecutado")

In [44]:
#Pasamos la imagen a texto usando la API de OCR.space
def ocr_space_ocr(ruta_imagen, idioma="spa"):
    """API gratuita de OCR.space - 500 peticiones/mes gratis"""
    url = "https://api.ocr.space/parse/image"
    
    with open(ruta_imagen, "rb") as archivo:
        respuesta = requests.post(
            url,
            files={"filename": archivo},
            data={
                "apikey": "helloworld",  # API key gratuita para pruebas
                "language": idioma,       # spa para español
                "isOverlayRequired": False
            }
        )
    if respuesta.status_code == 200:
        resultado = respuesta.json()
        return resultado.get("ParsedResults", [{}])[0].get("ParsedText", "")
texto = ocr_space_ocr("Fotos/foto.jpg", idioma="spa")


In [45]:
# Buscar el codigo nacional en la variable texto generado por OCR.space

patron = '\d{6}\.\d'
codigo_nacional = re.search(patron, texto) 
codigo_nacional = codigo_nacional.group() #aqui basicamente con .group() extraemos el texto que coincide con el patrón encontrado por re.search() 
codigo_nacional = codigo_nacional.split('.')[0] #Quitamos el punto, ya que la API no lo reconoce con el punto
print(f"Código nacional encontrado: {codigo_nacional}")


Código nacional encontrado: 661250


<>:3: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:3: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
C:\Users\alexx\AppData\Local\Temp\ipykernel_46488\2448673689.py:3: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
  patron = '\d{6}\.\d'


In [ ]:
# Buscar el codigo nacional en el resultado.txt generado por Tesseract
# carpeta_fotos = os.path.join(BASE_DIR, "Fotos")
# print(f"Buscando en carpeta: {carpeta_fotos}")
# archivo_txt = os.path.join(carpeta_fotos, "resultado.txt")

# # Sacamos el codigo nacional del archivo resultado.txt

# def extraer_numero(archivo_txt):
#     try:
#         with open(archivo_txt, 'r', encoding='utf-8') as archivo:
#             contenido = archivo.read()
#         patron = '\d{6}\.\d'
#         codigo_nacional = re.search(patron, contenido)
        
#         if codigo_nacional:
#             codigo_nacional = codigo_nacional.group()
#             print(f"Código nacional encontrado: {codigo_nacional}")
#             codigo_nacional = codigo_nacional.split('.')[0] #Quitamos el punto, ya que la API no lo reconoce con el punto
#             print(f"Código nacional sin punto: {codigo_nacional}")
#             os.remove(archivo_txt) # Eliminamos el archivo resultado.txt después de extraer el código nacional
#             print(f"Archivo '{os.path.basename(archivo_txt)}' eliminado correctamente de /pdfs.")
#             return codigo_nacional
#         else:
#             print("No se encontró el patrón en el archivo")
#             return None
        
#     except FileNotFoundError:
#         print(f"Error: No se encontró el archivo {archivo_txt}")
#         print(f"Buscando en: {os.path.abspath(archivo_txt)}")
#         return None
# if __name__ == "__main__":
#     print(f"Buscando archivo en: {archivo_txt}")
#     codigo_nacional = extraer_numero(archivo_txt)
    
#     if codigo_nacional:
#         print(f"Variable guardada: {codigo_nacional}")

In [46]:
# Hacemos la petición a la API de la AEMPS con el código nacional extraído
# codigo_nacional = "504665"

url = f"https://cima.aemps.es/cima/rest/presentacion/{codigo_nacional}"
respuesta = requests.get(url)

if respuesta.status_code == 200:
    datos = respuesta.json()
    nombre_medicamento = datos.get('nombre')
    print(f"Nombre: {nombre_medicamento}")
    for doc in datos['docs']:
        if doc['tipo'] == 1:
            try:
                url_ficha_tecnica = doc['url']
            except Exception as e:
                print(f"Ocurrió un error a la hora de guardar la url del PDF: {e}")
        elif doc['tipo'] == 2:
            try:
                url_prospecto= doc['url']
            except Exception as e:
                print(f"Ocurrió un error a la hora de guardar la url del PDF: {e}")
else:
    print(f"Error: No se encontró el medicamento (Status: {respuesta.status_code})")




Nombre: CRISTALMINA 10 mg/ml SOLUCION PARA PULVERIZACION CUTANEA, 1 frasco de 25 ml


In [50]:
# Crear la carpeta "data" si no existe y guardar los archivos PDF descargados
CARPETA_DESTINO = os.path.join(BASE_DIR, "data")

if not os.path.exists(CARPETA_DESTINO):
    os.makedirs(CARPETA_DESTINO)
    print(f"Carpeta creada en: {CARPETA_DESTINO}")

print("Descargando archivos...")

respuesta = requests.get(url_ficha_tecnica)
respuesta2 = requests.get(url_prospecto)
nombre_medicamento = nombre_medicamento.split()[0] #Esto se hace pk si es muy grande el nombre no entra como nombre del pdf
if respuesta.status_code == 200 and respuesta2.status_code == 200:
    nombre_ficha_tecnica = os.path.join(CARPETA_DESTINO, f"Ficha_{nombre_medicamento}.pdf")
    nombre_prospecto = os.path.join(CARPETA_DESTINO, f"Prospecto_{nombre_medicamento}.pdf")
    
    with open(nombre_ficha_tecnica, "wb") as archivo:
        archivo.write(respuesta.content)
    print(f"¡Hecho! Guardado en: {nombre_ficha_tecnica}")
    
    with open(nombre_prospecto, "wb") as archivo:
        archivo.write(respuesta2.content)
    print(f"¡Hecho! Guardado en: {nombre_prospecto}")
else:
    print(f"Error al descargar. Status codes: {respuesta.status_code}, {respuesta2.status_code}")


Descargando archivos...
¡Hecho! Guardado en: c:\Users\alexx\ia_identificar_medicamentos\data\Ficha_CRISTALMINA.pdf
¡Hecho! Guardado en: c:\Users\alexx\ia_identificar_medicamentos\data\Prospecto_CRISTALMINA.pdf


In [ ]:
# # Eliminar los archivos PDF descargados
# archivos = [nombre_ficha_tecnica, nombre_prospecto]


# for archivo in archivos:
#     if os.path.exists(archivo):
#         os.remove(archivo)
#         print(f"Archivo '{os.path.basename(archivo)}' eliminado correctamente de /pdfs.")
#     else:
#         print(f"El archivo '{os.path.basename(archivo)}' no existía en la carpeta.")